### Weekly Contest 393



#### 3114. Latest Time You Can Obtain After Replacing Characters

> You are given a string s representing a 12-hour format time where some of the digits (possibly none) are replaced with a "?".
>
> 12-hour times are formatted as "HH:MM", where HH is between 00 and 11, and MM is between 00 and 59. The earliest 12-hour time is 00:00, and the latest is 11:59.
>
> You have to replace all the "?" characters in s with digits such that the time we obtain by the resulting string is a valid 12-hour format time and is the latest possible.
> 
> Return the resulting string.

*Tags: Math*

Solution: Note it's 12-hour not 24-hour. Simple if else can solve the problem. Be careful with edge cases.

In [4]:
from typing import List
class Solution:
    def findLatestTime(self, s: str) -> str:
        s = list(s)

        if s[0] == s[1] == '?':
            s[0] = s[1] = '1'
        elif s[0] == '0':
            if s[1] == '?': s[1] = "9"
        elif s[0] == '1':
            if s[1] == '?': s[1] = "1"
        elif s[0] == '?':
            if s[1] == '0' or s[1] == '1': s[0] = '1'
            else: s[0] = '0'
                
        if s[3] == '?' and s[4] == '?':
            s[3], s[4] = '5', '9'
        elif s[3] == '?':
            s[3] = '5'
        elif s[4] == '?':
            s[4] = '9'
        
        return "".join(s)

#### 3115. Maximum Prime Difference

> You are given an integer array nums.
>
> Return an integer that is the maximum distance between the indices of two (not necessarily different) prime numbers in nums.

*Tags: Math*

Solution: Find the leftmost and rightmost prime numbers and compute their distance.

In [5]:
class Solution:
    def maximumPrimeDifference(self, nums: List[int]) -> int:
        ans = set()
        def judge(n):
            nonlocal ans
            for i in range(2, n):
                if n % i == 0: return
            ans.add(n)
        for i in range(2, 101): judge(i)
                        
        left, right = -1, -1
        for i in range(0, len(nums)):
            if nums[i] in ans: 
                left = i
                break
                
        for i in range(len(nums)-1, -1, -1):
            if nums[i] in ans: 
                right = i
                break
        return right - left

#### 3116. Kth Smallest Amount With Single Denomination Combination

> You are given an integer array coins representing coins of different denominations and an integer k.
>
> You have an infinite number of coins of each denomination. However, you are not allowed to combine coins of different denominations.
> 
> Return the kth smallest amount that can be made using these coins.

*Tags: Math, Binary Search*

Solution: This one is hard regarding the fact k could range from (1, 2*10**9), so we have to use an alternate way to think about it. First We need to use *binary search* to find out *tagret*. Here target has the property that the total number of valid denominations that are less or equal to target should equal to k. Another difficulty in the question is that we may count some combination twice, for example if coins = [3, 6, 9] then 3 and 6 can produce a denomination of 6. The duplicates will only happen when the domination is a multiple of their least common multiple. So count(a or b) = count(a) + count(b) - count(lcm(a, b)). When their are more coins that the formula will become more complicated. count(a or b or c) = count(a) + count(b) + count(c) - count(lcm(a, b)) - count(lcm(a, c)) - count(lcm(b, c)) + count(lcm(a, b, c)).

In [6]:
class Solution:
    
    def compute(self, coins, k):
        ans = []
        def getDivideNumber(idx, k, number):   
            if k == 0: 
                ans.append(number)
                return
            if idx >= len(coins): return
            getDivideNumber(idx+1, k, number) 
            getDivideNumber(idx+1, k-1, self.lcm(number, coins[idx]))
             
        getDivideNumber(0, k, 1)
        
        return ans

    def gcd(self, a, b):
        if b == 0: return a;
        return self.gcd(b, a % b);
        
    def lcm(self, a, b):
        return (a / self.gcd(a, b)) * b;
    
    def findKthSmallest(self, coins: List[int], k: int) -> int:
        coins.sort()

        lo, hi = min(coins), min(coins) * k
        divides = []
        for i in range(len(coins)):
            divides.append(self.compute(coins, i+1))
        
        while lo < hi:
            mid = (lo + hi) // 2
            cnt = 0
            for i in range(len(coins)):
                for d in divides[i]:
                    cnt += ((-1) ** (i % 2)) * (mid // d)
            if cnt < k:
                lo = mid + 1
            else:
                hi = mid
        return lo

#### 3108. Minimum Cost Walk in Weighted Graph

> You are given two arrays nums and andValues of length n and m respectively.
>
> The value of an array is equal to the last element of that array.
>
> You have to divide nums into m disjoint contiguous subarrays such that for the ith subarray [li, ri], the bitwise AND of the subarray elements is equal to andValues[i], in other words, nums[li] & nums[li + 1] & ... & nums[ri] == andValues[i] for all 1 <= i <= m, where & represents the bitwise AND operator.
>
> Return the minimum possible sum of the values of the m subarrays nums is divided into. If it is not possible to divide nums into m subarrays satisfying these conditions, return -1.

*Tags: Array, Dynamic Programming, Memorization*


Solution: When using the memorization, be careful of the total states you should store to avoid MLE. Avoid storing any variable which is not necessary. For example, here you don't need to store the total value from each subarray.

In [6]:
class Solution:
    def minimumValueSum(self, nums: List[int], andValues: List[int]) -> int:
        NUM_1 = int(pow(2,32)) - 1
        n, k = len(nums), len(andValues)
        
        ans = inf

        @lru_cache(None)
        def dfs(idx, remaining_k, curAndValue):
            nonlocal ans
            if idx == len(nums) and remaining_k == 0:
                return 0
            if idx >= len(nums): return inf
            if remaining_k == 0 and idx < len(nums): return inf

            cur = inf
            if curAndValue & nums[idx] == andValues[k-remaining_k]:
                cur = nums[idx] + dfs(idx+1, remaining_k-1, NUM_1)
            cur = min(cur, dfs(idx+1, remaining_k, curAndValue & nums[idx]))
            return cur
            
        ans = dfs(0, k, NUM_1)
        return ans if ans != inf else -1